In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()

In [3]:
# 단어장(vocabulary) 만들기
vocab = list(set(sentence))
print(vocab)

['memory', 'Repeat', 'medicine', 'is', 'the', 'for', 'best']


In [4]:
# 단어장의 단어에 고유한 정수 인덱스 부여 // 모르는 단어 의미하는 UNK 토큰 추가
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0

In [5]:
print(word2index)

{'memory': 1, 'Repeat': 2, 'medicine': 3, 'is': 4, 'the': 5, 'for': 6, 'best': 7, '<unk>': 0}


In [6]:
print(word2index['memory'])

1


In [8]:
# 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'memory', 2: 'Repeat', 3: 'medicine', 4: 'is', 5: 'the', 6: 'for', 7: 'best', 0: '<unk>'}


In [10]:
print(index2word[2])

Repeat


In [13]:
# 정수 2 와 맵핑되는 단어는 Repeat
# 데이터의 각 단어를 정수로 인코딩하는 동시에, 입력 데이터와 레이블 데이터를 만드는 build_data라는 함수 생성

def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [14]:
X, Y = build_data(sentence, word2index)
print(X)
print(Y)

tensor([[2, 4, 5, 7, 3, 6]])
tensor([[4, 5, 7, 3, 6, 1]])


In [16]:
# 모델 구현
# 이전 모델들과는 임베딩 층을 추가 함으로써 차별점을 둠
# 임베딩 층은 크게 두 가지 인자를 받는데 첫번째 인자는 단어장의 크기, 두번째 인자는 임베딩 백터의 차원


class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                           embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                               batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) #출력은 원-핫 벡터 크기를 가져야 함 // 또는 집합 크기만큼 가져야함
        
    def forward(self, x):
        # 1. 임베딩 층
        # 크기 변화 : (배치 크기, 시퀸스 길이) => (배치 크기, 시퀸스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기 변화 : (배치 큭, 시퀸스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀸스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기 변화 : (배치 크기, 시퀸스 길이, 은닉층 크기) => (배치 크기, 시퀸스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기 변화 : (배치 크기, 시퀸스 길이, 단어장 크기) => (배치 크기*시퀸스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [17]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용 // unk 토큰을 크기에 포함
input_size = 5 # 임베딩 된 차원으 크기 및 RNN 층 입력 차원의 크기 5
hidden_size = 20 # RNN 은닉층 크기 20

In [18]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)

# 손실 함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제 값은 원-핫 인코딩 안해도 됨

# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())